In [1]:
# direct to proper path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
import pickle
from prettytable import PrettyTable

from codes.Q_BAI import Q_UGapEb, Q_SAR, uniform_sampling
from codes.Environment import Mixture_AbsGau, setup_env, Exp
from codes.SimulatedGames import simulate
from codes.plots import plot_hist, plot_eva

%matplotlib inline

In [2]:
# setting

num_exper = 50
num_rounds = 10000
num_arms = 10
est_flag = True # estimate the lower bound of hazard rate L
tau = 0.9 # quantile level

# environment

environments = [
    {Mixture_AbsGau: [[3, 1, 0,1, 1], [1.2, 4, 0, 1, 1], 
                      [3, 1, 0,1, 1], [1.2, 4, 0, 1, 1],
                      [3, 1, 0,1, 1], [1.2, 4, 0, 1, 1],
                      [3, 1, 0,1, 1], [1.2, 4, 0, 1, 1],
                      [3, 1, 0,1, 1]],
     Exp: [1/3.5]} # mu1, sigma1, mu2, sigma2, p
    
]


rewards_env, quantiles, Ls, means, samples = setup_env(num_arms, environments, tau)

{<class 'codes.Environment.Mixture_AbsGau'>: [[3, 1, 0, 1, 1], [1.2, 4, 0, 1, 1], [3, 1, 0, 1, 1], [1.2, 4, 0, 1, 1], [3, 1, 0, 1, 1], [1.2, 4, 0, 1, 1], [3, 1, 0, 1, 1], [1.2, 4, 0, 1, 1], [3, 1, 0, 1, 1]], <class 'codes.Environment.Exp'>: [0.2857142857142857]}


In [3]:
quantiles

defaultdict(list,
            {'Mixture_AbsGau_[[3, 1, 0, 1, 1], [1.2, 4, 0, 1, 1], [3, 1, 0, 1, 1], [1.2, 4, 0, 1, 1], [3, 1, 0, 1, 1], [1.2, 4, 0, 1, 1], [3, 1, 0, 1, 1], [1.2, 4, 0, 1, 1], [3, 1, 0, 1, 1]]Exp_[0.2857142857142857]': [4.285365110890041,
              6.806551781875078,
              4.267796844197339,
              6.908504357172303,
              4.276253197671883,
              6.886688009725967,
              4.285903606359307,
              6.845199098290813,
              4.273981603442657,
              8.144235308065943]})

In [4]:
"""

arm_name_dict = {
    0: 'A',
    1: 'B',
    2: 'C'
}

for key in quantiles.keys():
    print(key)
    quantiles[key] = list(np.around(np.array(quantiles[key]),2))
    means[key] = list(np.around(np.array(means[key]),2))
    t = PrettyTable(['Eva', 'A', 'B', 'C', 'Best Arm'])
    t.add_row([str(tau) + '-Quantile'] + quantiles[key]+ [arm_name_dict[np.argmax(quantiles[key])]])
    t.add_row(['Mean']+ means[key]+ [arm_name_dict[np.argmax(means[key])]])
    t.add_row(['L']+ Ls[key]+ [arm_name_dict[np.argmax(Ls[key])]])
    print(t)
    print()
"""

"\n\narm_name_dict = {\n    0: 'A',\n    1: 'B',\n    2: 'C'\n}\n\nfor key in quantiles.keys():\n    print(key)\n    quantiles[key] = list(np.around(np.array(quantiles[key]),2))\n    means[key] = list(np.around(np.array(means[key]),2))\n    t = PrettyTable(['Eva', 'A', 'B', 'C', 'Best Arm'])\n    t.add_row([str(tau) + '-Quantile'] + quantiles[key]+ [arm_name_dict[np.argmax(quantiles[key])]])\n    t.add_row(['Mean']+ means[key]+ [arm_name_dict[np.argmax(means[key])]])\n    t.add_row(['L']+ Ls[key]+ [arm_name_dict[np.argmax(Ls[key])]])\n    print(t)\n    print()\n"

In [5]:
# plot_hist(samples)

In [6]:
# run experiments

def run_exper(rewards_env, hyperpara_list, num_exper, budget_or_confi, epsilon, tau, m, policy, summary_stats, est_flags = [est_flag]):
    for key in rewards_env.keys():
        for hyperpara in hyperpara_list:
            for est_flag in est_flags:
                name = key + '_' + str(num_exper) + '_' + str(num_rounds)
                policy_name = str(policy).split('.')[-1].split('\'')[0] + '-'
                subname = policy_name + str(hyperpara)
                print(name + subname)
                p = IntProgress(max = num_exper)
                p.description = 'Running'
                display(p)
                error, std = simulate(rewards_env[key],summary_stats[key], policy, 
                                                 epsilon, tau, m, budget_or_confi, num_exper, est_flag, hyperpara, None, p)
                return error, std

In [7]:
budget = 5000
epsilon = 0.5
tau = 0.9
m = 1
num_exper = 100

In [8]:
hyperpara_list = [[1]]
error, std = run_exper(rewards_env, hyperpara_list, num_exper, budget, epsilon, tau, m, Q_SAR, quantiles)

Mixture_AbsGau_[[3, 1, 0, 1, 1], [1.2, 4, 0, 1, 1], [3, 1, 0, 1, 1], [1.2, 4, 0, 1, 1], [3, 1, 0, 1, 1], [1.2, 4, 0, 1, 1], [3, 1, 0, 1, 1], [1.2, 4, 0, 1, 1], [3, 1, 0, 1, 1]]Exp_[0.2857142857142857]_100_10000Q_SAR-[1]


IntProgress(value=0, description='Running')

In [9]:
error

0.17

In [10]:
std

0.375632799419859